# Projet d'Optimisation - Rangement d'un tuyau de jardin

# Enol Alvarez & Louis-Justin Tallot

## 1. Etude du problème d'optimisation

#### Q1) Interpréter le coût (4). Que cherche-t-on à minimiser ?

On cherche $min \left(K\left(x_n - \dfrac{L}{2}\right)^2 + \text{d}x^2 \sum \limits _{i=0}^N y_i^2   \right)$

-> distance à l'horizontale + distance au parterre de fleurs

#### Q2) Pourquoi n'a-t-on pas plutôt imposé la contrainte $x_N = L/2$ ?